In [1]:
#%pip install -q huggingface_hub
#%pip install -q datasets
#%pip install -q transformers
#%pip install -q accelerate
#%pip install -q gradio
#%pip install -q scikit-learn

In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/SampleSubmission.csv')

In [3]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [5]:
duplicates = train.duplicated(subset=['safe_text'])
print(f"Number of duplicates in safe_text col:{duplicates.sum()}")

Number of duplicates in safe_text col:343


In [6]:
#remove duplicates
train = train[~duplicates]
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9658 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   9658 non-null   object 
 1   safe_text  9658 non-null   object 
 2   label      9657 non-null   float64
 3   agreement  9656 non-null   float64
dtypes: float64(2), object(2)
memory usage: 377.3+ KB


In [7]:
train['safe_text']

0        Me &amp; The Big Homie meanboy3000 #MEANBOY #M...
1        I'm 100% thinking of devoting my career to pro...
2        #whatcausesautism VACCINES, DO NOT VACCINATE Y...
3        I mean if they immunize my kid with something ...
4        Thanks to <user> Catch me performing at La Nui...
                               ...                        
9996     Living in a time where the sperm I used to was...
9997     <user> <user>  In spite of all measles outbrea...
9998     Interesting trends in child immunization in Ok...
9999     CDC Says Measles Are At Highest Levels In Deca...
10000    Pneumonia vaccine: for women w risk of pulmona...
Name: safe_text, Length: 9658, dtype: object

In [8]:
train['safe_text']

0        Me &amp; The Big Homie meanboy3000 #MEANBOY #M...
1        I'm 100% thinking of devoting my career to pro...
2        #whatcausesautism VACCINES, DO NOT VACCINATE Y...
3        I mean if they immunize my kid with something ...
4        Thanks to <user> Catch me performing at La Nui...
                               ...                        
9996     Living in a time where the sperm I used to was...
9997     <user> <user>  In spite of all measles outbrea...
9998     Interesting trends in child immunization in Ok...
9999     CDC Says Measles Are At Highest Levels In Deca...
10000    Pneumonia vaccine: for women w risk of pulmona...
Name: safe_text, Length: 9658, dtype: object

In [9]:
#function to clean the text data
#import re
#from nltk.corpus import stopwords
#import nltk

# Ensure the stopwords dataset is downloaded
#nltk.download('stopwords')

#def clean_text(text):
    # Remove unwanted characters and symbols
    #text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove URLs
    #text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Remove extra spaces
    #text = text.replace('\n', ' ').replace('\r', ' ')
    # Remove leading and trailing spaces
    #text = text.strip()
    # Convert to lowercase
    #text = text.lower()
    # Remove special characters 
    #text = re.sub(r'\W+', ' ', text)

    #return text

#### Transformers using RoBERTa

##### The RoBERTa tokenizer is a subword tokenizer that uses byte pair encoding (BPE) to handle out-of-vocabulary words.    

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\Owner\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train.sample(5)

,tweet_id,safe_text,label,agreement
7344,XTR1EXUG,<user> Let the Science Speak: The Evidence Sup...,-1.0,0.666667
9867,63DW3AZ0,<user> #RoommatesIn5Words\nHave you ever had m...,0.0,1.000000
5632,E4RZNIY5,More mumps cases confirmed on UW campus. Check...,1.0,1.000000
1890,PQ8LGTN6,<user> <user> <user> No to required Immunizati...,0.0,0.666667
2158,5X6OQLEQ,RT <user> Kids and Teens Get Free Immunization...,0.0,0.666667
